# Examining Hacker News Site to Determine Best Time to Post for Most Comments
***
In this project, we are working with the data set submissions of popular technology site [Hacker News](https://news.ycombinator.com/).
Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.
You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. 
Below are descriptions of the columns:
- `id`: The unique identifier from Hacker News for the post
- `title`: The title of the post
- `url`: The URL that the posts links to, if the post has a URL
- `num_points`: The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes 
- `num_comments`: The number of comments that were made on the post
- `author`: The username of the person who submitted the post
- `created_at`: The date and time at which the post was submitted

We're specifically interested in posts whose titles begin with either `Ask HN` or `Show HN`. Users submit `Ask HN` posts to ask the Hacker News community a specific question. Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just generally something interesting.

## Our goal:
We'll compare these two types of posts to determine the following:
- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average?


## Part One: Opening the Data
Let's start by importing the libraries we need and reading the data set into a list of lists.

In [1]:
from csv import reader

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


Remove the header row to make it possible to analyze our data.

In [3]:
headers = hn[0]
hn = hn[1:]
print(headers)
print()
print(hn[:5])

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']

[['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12'], ['10482257', 'Title II kills investment? Comcast and other ISPs are now spending more', 'http

## Part Two: Data Cleaning
We are only interested in posts that have titles beginning with `Ask HN` and `Show HN`, we'll create new lists of lists containing just the data for those titles. 

To find the posts that begin with either `Ask HN` or `Show HN`, we'll use the string method `startswith`.

In [4]:
# Lists to store Ask HN, Show HN, and other posts.
ask_posts = []
show_posts = []
other_posts = []

# loop through the hn data, without the header, and assign title to title variable
for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith("ask hn"):
        ask_posts.append(row)
    elif title.startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)

print(len(ask_posts), '\n')
print(len(show_posts), '\n')
print(len(other_posts), '\n')

1744 

1162 

17193 



In [5]:
# the first five rows of the `Ask HN` posts
print(ask_posts[:5])

[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14'], ['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20'], ['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']]


In [6]:
# the first five rows of the `Show HN` posts
print(show_posts[:5])

[['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46'], ['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', '1', 'h8liu', '4/28/2016 18:05'], ['12178806', 'Show HN: Webscope  Easy way for web developers to communicate with Clients', 'http://webscopeapp.com', '3', '3', 'fastbrick', '7/28/2016 7:11'], ['10872799', 'Show HN: GeoScreenshot  Easily test Geo-IP based web pages', 'https://www.geoscreenshot.com/', '1', '9', 'kpsychwave', '1/9/2016 20:45']]


## Part three: Data Analysis
Now, let's determine if ask posts or show posts receive more comments on average.

In [21]:
# total comments are
total_comments = 0
for row in hn:
    num_comments = int(row[4])
    total_comments += num_comments
    
print ("Total number of comments on all posts is :", total_comments)

# the total number of comments in ask posts
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
    
print("Total number of comments on Ask HN posts is :", total_ask_comments)
avg_ask_comments = total_ask_comments / total_comments
print("Average number ofcomments on Ask HN posts is: {0:.3f}".format(avg_ask_comments))

# the total number of comments in show posts
total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
    
print("Total number of comments on Ask HN posts is: ", total_show_comments)
avg_show_comments = total_show_comments / total_comments
print("Average number ofcomments on Ask HN posts is: {0:.3f}".format(avg_show_comments))

difference_of_avgs = avg_ask_comments - avg_show_comments
print("The difference between the two averages is {0:.3f}".format(difference_of_avgs))

Total number of comments on all posts is : 498474
Total number of comments on Ask HN posts is : 24483
Average number ofcomments on Ask HN posts is: 0.049
Total number of comments on Ask HN posts is:  11988
Average number ofcomments on Ask HN posts is: 0.024
The difference between the two averages is 0.025


The average number of comments on `Ask HN` is `0.049` while that of `Show HN` is `0.024`. This shows that, the posts on `Ask HN` gets more comments compared to the posts on `Show HN`. 
the difference between the two averages is 0.025. The `Ask HN` comments are `0.025` which is `2.5%` more than the `Show HN`. 

On average, ask posts receive more comments than show posts. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:
- Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.

Calculating the amount of ask posts and comments by hour created. We'll use the datetime module to work with the data in the created_at column.

In [31]:
# import datetime 
import datetime as dt

# list to store results
result_list = []

# iterate over the ask posts and append the post row number of comments and time of creation to result list
for post in ask_posts:
    created_at = post[6]
    num_comments = int(post[4])
    result_list.append([created_at, num_comments])

counts_by_hour = {}
comments_by_hour = {}

# loop through the rows of result_list and get the hour 
for row in result_list:
    hour = row[0]
    hour = dt.datetime.strptime(hour, "%m/%d/%Y %H:%M")  # returns a datetime object
    hour  = hour.strftime("%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1  
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1  
        comments_by_hour[hour] += row[1]


print("The number of ask posts created during each hour of the day.  \n", counts_by_hour)
print()
print("The corresponding number of comments received for ask posts created at each hour. \n", comments_by_hour)


The number of ask posts created during each hour of the day.  
 {'15': 116, '22': 71, '17': 100, '16': 108, '23': 68, '20': 80, '14': 107, '05': 46, '21': 109, '18': 109, '19': 110, '10': 59, '03': 54, '07': 34, '01': 60, '11': 58, '06': 44, '08': 48, '13': 85, '02': 58, '04': 47, '09': 45, '12': 73, '00': 55}

The corresponding number of comments received for ask posts created at each hour. 
 {'15': 4477, '22': 479, '17': 1146, '16': 1814, '23': 543, '20': 1722, '14': 1416, '05': 464, '21': 1745, '18': 1439, '19': 1188, '10': 793, '03': 421, '07': 267, '01': 683, '11': 641, '06': 397, '08': 492, '13': 1253, '02': 1381, '04': 337, '09': 251, '12': 687, '00': 447}


Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day.  
We are going to create a `list of lists` containing the hours during which posts were created and average number of comments those posts received.

In [32]:
# using list comprehension
avg_by_hour = [[h, comments_by_hour[h]/counts_by_hour[h]] for h in counts_by_hour]
print(avg_by_hour)

[['15', 38.5948275862069], ['22', 6.746478873239437], ['17', 11.46], ['16', 16.796296296296298], ['23', 7.985294117647059], ['20', 21.525], ['14', 13.233644859813085], ['05', 10.08695652173913], ['21', 16.009174311926607], ['18', 13.20183486238532], ['19', 10.8], ['10', 13.440677966101696], ['03', 7.796296296296297], ['07', 7.852941176470588], ['01', 11.383333333333333], ['11', 11.051724137931034], ['06', 9.022727272727273], ['08', 10.25], ['13', 14.741176470588234], ['02', 23.810344827586206], ['04', 7.170212765957447], ['09', 5.5777777777777775], ['12', 9.41095890410959], ['00', 8.127272727272727]]


Although we now have the results we need, this format makes it hard to identify the hours with the highest values. 
Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [33]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
print(swap_avg_by_hour)

[[38.5948275862069, '15'], [6.746478873239437, '22'], [11.46, '17'], [16.796296296296298, '16'], [7.985294117647059, '23'], [21.525, '20'], [13.233644859813085, '14'], [10.08695652173913, '05'], [16.009174311926607, '21'], [13.20183486238532, '18'], [10.8, '19'], [13.440677966101696, '10'], [7.796296296296297, '03'], [7.852941176470588, '07'], [11.383333333333333, '01'], [11.051724137931034, '11'], [9.022727272727273, '06'], [10.25, '08'], [14.741176470588234, '13'], [23.810344827586206, '02'], [7.170212765957447, '04'], [5.5777777777777775, '09'], [9.41095890410959, '12'], [8.127272727272727, '00']]


In [34]:
# sorted() function to sort swap_avg_by_hour in descending order.
# Since the first column of this list is the average number of comments, 
# sorting the list will sort by the average number of comments.
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
print(sorted_swap)

[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [13.20183486238532, '18'], [11.46, '17'], [11.383333333333333, '01'], [11.051724137931034, '11'], [10.8, '19'], [10.25, '08'], [10.08695652173913, '05'], [9.41095890410959, '12'], [9.022727272727273, '06'], [8.127272727272727, '00'], [7.985294117647059, '23'], [7.852941176470588, '07'], [7.796296296296297, '03'], [7.170212765957447, '04'], [6.746478873239437, '22'], [5.5777777777777775, '09']]


In [37]:
# Top 5 Hours to post Ask Posts for the highest number Comments
print("Top 5 Hours for Ask Posts Comments \n")
# Loop through each average and each hour
for row in sorted_swap[:5]:
    hour = dt.datetime.strptime(row[1], "%H")
    hour = hour.strftime("%H:%M")
    print("{h}: {c:.2f} average comments per post".format(h=hour, c=row[0]))
print()
print()
    
print("The worst 5 Hours for Ask Posts Comments \n")
# Loop through each average and each hour
for row in sorted_swap[-5:]:
    hour = dt.datetime.strptime(row[1], "%H")
    hour = hour.strftime("%H:%M")
    print("{h}: {c:.2f} average comments per post".format(h=hour, c=row[0]))



Top 5 Hours for Ask Posts Comments 

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


The worst 5 Hours for Ask Posts Comments 

07:00: 7.85 average comments per post
03:00: 7.80 average comments per post
04:00: 7.17 average comments per post
22:00: 6.75 average comments per post
09:00: 5.58 average comments per post


## Conclusion
The best time to create a post for Hackers News is around `15:00 PM`. The other best times is `16:00 PM, 20:00 PM and 21:00PM`. The best time in the morning to post on hackers News is at `2:00 AM`. During this time, the average number of comments on posts is the highest compared to other hours of the day.
The best time to post to get the most comments are at 15, 2, and 20 UTC.